In [21]:
from src.lightning_modules import FM
from src.networks import StableDiffusionXL, MediumUNet

unet = MediumUNet()
encoder = StableDiffusionXL()

fm = FM(model=unet, encoder_decoder=encoder)

Loaded VAE model stabilityai/stable-diffusion-xl-base-1.0


In [1]:
import IPython.display as ipd
from src.dataset import VoxCeleb, EarsGender, GenderAudioDataset
import random

gender = 'male'

vox_dataset = VoxCeleb(gender=gender)
print("Length of VoxCeleb dataset: ", len(vox_dataset))

ears_dataset = EarsGender(gender=gender)
print("Length of Ears dataset: ", len(ears_dataset))

gender_dataset = GenderAudioDataset(gender=gender)
print("Length of gender dataset: ", len(gender_dataset))

for _ in range(5):
    rand_idx = random.randint(0, len(vox_dataset))
    gender_sample = gender_dataset[rand_idx]
    ipd.display(ipd.Audio(gender_sample, rate=gender_dataset.sample_rate))

Length of VoxCeleb dataset:  3682
Length of Ears dataset:  6450
Length of gender dataset:  10132


In [5]:
from src.networks import PretrainedMimi
from src.dataset import EarsGender
import torchaudio
import IPython.display as ipd
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
dataset = EarsGender(gender='male')
x = dataset[0].unsqueeze(0).to(device)
encoder = PretrainedMimi().to(device)
encoded = encoder.encode(x)
decoded = encoder.decode(encoded).detach()


ipd.display(ipd.Audio(x.reshape(1, -1), rate=24000))
ipd.display(ipd.Audio(decoded.reshape(1, -1), rate=24000))

/work3/s214630/Latent-DSB/.venv/lib/python3.12/site-packages/transformers/models/mimi/modeling_mimi.py:164: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.register_buffer("padding_total", torch.tensor(kernel_size - stride, dtype=torch.int64), persistent=False)


Loaded Mimi model
